In [238]:
import pandas as pd
import numpy as np

# Laden der CSV-Dateien in DataFrames
df1 = pd.read_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/test_data.csv', usecols=['date', 'n_duty'])
df2 = pd.read_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/prediction_MLR_calls.csv', usecols=['date', 'MLR'])
df3 = pd.read_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/prediction_Sick-ratioLR.csv', usecols=['date', 'Lasso Regression'])
df4 = pd.read_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/test_data.csv', usecols=['date', 'dafted', 'sby_need'])

df5 = pd.read_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/prediction_Ridge_Regression_calls.csv', usecols=['date', 'Ridge_Regression'])


# Umbenennen der Spalten, um Namenskonflikte zu vermeiden
df2.rename(columns={'MLR': 'calls'}, inplace=True)
df3.rename(columns={'Lasso Regression': 'sick_ratio'}, inplace=True)

# Hinzufügen der neuen Spalte 'Standardwert' zu df2
df2['Standardwert'] = 90

# Mergen der DataFrames anhand der 'date' Spalte
merged_df = pd.merge(df1, df2, on='date')
merged_df = pd.merge(merged_df, df3, on='date')

# Berechnen der neuen Spalte 'prediction_sby_needed'

merged_df['prediction_sby_needed'] = merged_df['calls'] / 5 - merged_df['n_duty'] + merged_df['n_duty'] * merged_df['sick_ratio']

merged_df['prediction_sby_needed'] = df5['Ridge_Regression']-1900 

# Erstellen eines neuen DataFrames mit 'date', 'prediction_sby_needed' und 'Standardwert'
result_df = merged_df[['date', 'prediction_sby_needed', 'n_duty', 'sick_ratio', 'calls']]
result_df['prediction_adjusted'] = result_df['prediction_sby_needed'].apply(lambda x: max(x, 0))


# Hinzufügen der Spalte 'actuals' zum resultierenden DataFrame
# Es wird davon ausgegangen, dass df4 die gleichen 'date'-Werte wie merged_df hat
result_df = pd.merge(result_df, df4[['date', 'dafted', 'sby_need']], on='date')
result_df['actuals'] = result_df['sby_need']

# Berechnung des RMSE basierend auf den Prognosen
rmse = np.sqrt(np.mean((result_df['actuals'] - result_df['prediction_adjusted'])**2))
print(f'Root Mean Squared Error (RMSE) der Prognosen: {rmse:.2f}')
AbsoluteError = (result_df['actuals'] - result_df['prediction_adjusted'])
print(f'Absolut Error der Prognose: {AbsoluteError}')

result_df2 = merged_df[['date']]
result_df['prediction_adjusted'] = result_df['prediction_sby_needed'].apply(lambda x: max(x, 0))
result_df2['actuals'] = result_df['dafted'] + result_df['sby_need']
result_df2.to_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/results.csv', index=False)

# Berechnung des RMSE für den Standardwert 90
rmse_standard = np.sqrt(np.mean((result_df['actuals'] - 90)**2))
AbsoluteErrorStd = (result_df['actuals'] - 90)
print(f'Root Mean Squared Error (RMSE) für den Standardwert: {rmse_standard:.2f}')
print(f'Absolut Error für den standardwert: {AbsoluteErrorStd}')

# Berechnung des Verhältnisses Actuals/Prognose, mit 100% für Prognosen von 0
result_df['ratio_prediction_adjusted'] = result_df.apply(
    lambda row: 100 if row['prediction_adjusted'] == 0 else (row['actuals'] / row['prediction_adjusted'] * 100),
    axis=1
)

# Berechnung des Durchschnitts des Verhältnisses
average_ratio_prediction_adjusted = result_df['ratio_prediction_adjusted'].mean()
print(f'Durchschnittliches Verhältnis (Actuals/Prognose): {average_ratio_prediction_adjusted:.2f}%')

# Berechnung des Verhältnisses Actuals/Standardwert 90, mit 100% für Prognosen von 0
result_df['ratio_standard'] = result_df.apply(
    lambda row: 100 if 90 == 0 else (row['actuals'] / 90 * 100),
    axis=1
)

# Berechnung des Durchschnitts des Verhältnisses mit Standardwert 90
average_ratio_standard = result_df['ratio_standard'].mean()
print(f'Durchschnittliches Verhältnis (Actuals/Standardwert 90): {average_ratio_standard:.2f}%')

Root Mean Squared Error (RMSE) der Prognosen: 102.51
Absolut Error der Prognose: 0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
       ...  
328    330.0
329      0.0
330      0.0
331      0.0
332      0.0
Length: 333, dtype: float64
Root Mean Squared Error (RMSE) für den Standardwert: 101.41
Absolut Error für den standardwert: 0      -90
1      -90
2      -90
3      -90
4      -90
      ... 
328    240
329    -90
330    -90
331    -90
332    -90
Name: actuals, Length: 333, dtype: int64
Durchschnittliches Verhältnis (Actuals/Prognose): 99.73%
Durchschnittliches Verhältnis (Actuals/Standardwert 90): 49.89%


/var/folders/wj/rgdct6rx7fjbryz5h_nwf3_c0000gn/T/ipykernel_45574/1138718461.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df2['actuals'] = result_df['dafted'] + result_df['sby_need']


In [235]:
import pandas as pd
import numpy as np

# Laden der CSV-Dateien in DataFrames
df1 = pd.read_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/test_data.csv', usecols=['date', 'n_duty'])
df2 = pd.read_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/prediction_MLR_calls.csv', usecols=['date', 'MLR'])
df3 = pd.read_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/prediction_Sick-ratioLR.csv', usecols=['date', 'Lasso Regression'])
df4 = pd.read_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/test_data.csv', usecols=['date', 'dafted', 'sby_need'])
df5 = pd.read_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/prediction_MLP_calls.csv', usecols=['date', 'MLP'])

# Umbenennen der Spalten, um Namenskonflikte zu vermeiden
df2.rename(columns={'MLR': 'calls'}, inplace=True)
df3.rename(columns={'Lasso Regression': 'sick_ratio'}, inplace=True)

# Hinzufügen der neuen Spalte 'Standardwert' zu df2
df2['Standardwert'] = 90

# Mergen der DataFrames anhand der 'date'-Spalte
merged_df = pd.merge(df1, df2, on='date')
merged_df = pd.merge(merged_df, df3, on='date')

# Berechnen der neuen Spalte 'prediction_sby_needed'
merged_df['prediction_sby_needed'] = merged_df['calls'] / 5 - merged_df['n_duty'] + merged_df['n_duty'] * merged_df['sick_ratio']

# Überschreiben der 'prediction_sby_needed' Spalte mit den Ridge Regression Werten
merged_df['prediction_sby_needed'] = df5['MLP'] - 1900 

# Erstellen eines neuen DataFrames mit den relevanten Spalten
result_df = merged_df[['date', 'prediction_sby_needed', 'n_duty', 'sick_ratio', 'calls']]

# Vorhersagen anpassen (keine negativen Werte)
result_df['prediction_adjusted'] = result_df['prediction_sby_needed'].apply(lambda x: max(x, 0))

# Hinzufügen der Spalte 'actuals' zum resultierenden DataFrame durch Mergen mit df4
result_df = pd.merge(result_df, df4[['date', 'dafted', 'sby_need']], on='date')
result_df['actuals'] = result_df['dafted'] + result_df['sby_need']

# Berechnung des RMSE basierend auf den Prognosen
rmse = np.sqrt(np.mean((result_df['actuals'] - result_df['prediction_adjusted'])**2))
print(f'Root Mean Squared Error (RMSE) der Prognosen: {rmse:.2f}')

# Berechnung des absoluten Fehlers
AbsoluteError = result_df['actuals'] - result_df['prediction_adjusted']
print(f'Absoluter Fehler der Prognose: {AbsoluteError}')


# Export der Ergebnisse in eine CSV-Datei
result_df2 = result_df[['date', 'actuals', 'prediction_adjusted']]

# Export der Ergebnisse in eine CSV-Datei
result_df2 = result_df[['date', 'actuals', 'prediction_adjusted']].copy()

# Umwandlung aller Spalten außer 'date' in Ganzzahlen
for col in result_df2.columns:
    if col != 'date':
        result_df2.loc[:, col] = result_df2[col].astype(int)

result_df2.to_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/results.csv', index=False)



# Berechnung des RMSE für den Standardwert 90
rmse_standard = np.sqrt(np.mean((result_df['actuals'] - 90)**2))


Root Mean Squared Error (RMSE) der Prognosen: 158.49
Absoluter Fehler der Prognose: 0        0.000000
1        0.000000
2        0.000000
3        0.000000
4        0.000000
          ...    
328    570.000000
329      0.000000
330    -35.028138
331      0.000000
332      0.000000
Length: 333, dtype: float64


In [ ]:
import pandas as pd
import numpy as np

# Laden der CSV-Dateien in DataFrames
df1 = pd.read_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/test_data.csv', usecols=['date', 'n_duty'])
df2 = pd.read_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/prediction_MLR_calls.csv', usecols=['date', 'MLR'])
df3 = pd.read_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/prediction_Sick-ratioLR.csv', usecols=['date', 'Lasso Regression'])
df4 = pd.read_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/test_data.csv', usecols=['date', 'dafted', 'sby_need'])
df5 = pd.read_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/prediction_MLP_calls.csv', usecols=['date', 'MLP'])

# Umbenennen der Spalten, um Namenskonflikte zu vermeiden
df2.rename(columns={'MLR': 'calls'}, inplace=True)
df3.rename(columns={'Lasso Regression': 'sick_ratio'}, inplace=True)

# Hinzufügen der neuen Spalte 'Standardwert' zu df2
df2['Standardwert'] = 90

# Mergen der DataFrames anhand der 'date'-Spalte
merged_df = pd.merge(df1, df2, on='date')
merged_df = pd.merge(merged_df, df3, on='date')

# Berechnen der neuen Spalte 'prediction_sby_needed'
merged_df['prediction_sby_needed'] = merged_df['calls'] / 5 - merged_df['n_duty'] + merged_df['n_duty'] * merged_df['sick_ratio']

# Überschreiben der 'prediction_sby_needed' Spalte mit den MLP Regression Werten
merged_df['prediction_sby_needed'] = df5['MLP'] - 1900 

# Erstellen eines neuen DataFrames mit den relevanten Spalten
result_df = merged_df[['date', 'prediction_sby_needed', 'n_duty', 'sick_ratio', 'calls']]

# Vorhersagen anpassen (keine negativen Werte)
result_df['prediction_adjusted'] = result_df['prediction_sby_needed'].apply(lambda x: max(x, 0))

# Hinzufügen der Spalte 'actuals' zum resultierenden DataFrame durch Mergen mit df4
result_df = pd.merge(result_df, df4[['date', 'dafted', 'sby_need']], on='date')
result_df['actuals'] = result_df['sby_need']

# Berechnung des RMSE basierend auf den Prognosen
rmse = np.sqrt(np.mean((result_df['actuals'] - result_df['prediction_adjusted'])**2))
print(f'Root Mean Squared Error (RMSE) der Prognosen: {rmse:.2f}')

# Berechnung des absoluten Fehlers
AbsoluteError = result_df['actuals'] - result_df['prediction_adjusted']
print(f'Absoluter Fehler der Prognose: {AbsoluteError}')

# Export der Ergebnisse in eine CSV-Datei
result_df2 = result_df[['date', 'actuals', 'prediction_adjusted']]

# Umwandlung aller Spalten außer 'date' in Ganzzahlen
for col in result_df2.columns:
    if col != 'date':
        result_df2.loc[:, col] = result_df2[col].astype(int)

result_df2.to_csv('/Users/frederic/Desktop/Fallstudie-Model-Engineering/data/processed/results.csv', index=False)

# Berechnung des RMSE für den Standardwert 90
rmse_standard = np.sqrt(np.mean((result_df['actuals'] - 90)**2))
print(f'Root Mean Squared Error (RMSE) für den Standardwert 90: {rmse_standard:.2f}')
